In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Pytorch version is: ", torch.__version__)
print("You are using: ", device)

Pytorch version is:  2.0.1+cu118
You are using:  cuda


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import re
import io
import tqdm
import json
import datetime
import argparse
import numpy as np
from itertools import chain, count
from collections import Counter, defaultdict
from babel.numbers import parse_decimal, NumberFormatError
import nltk
from nltk.corpus import wordnet


import torch
import torchtext.data
import torchtext.vocab
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import unicodedata
import six


In [ ]:
!pip install records
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openpyxl: filename=openpyxl-2.4.11-py2.py3-none-any.whl size=222821 sha256=f07f766d8f14a6fe81035a8adefdbc6fe8402b5c15179550c0d07bb5ff5db0a4
  Stored in directory: /root/.cache/pip/wheels/f6/0a/c7/3744d0f62b960ef9ff407e2c38975e058629ace1255e8bd95b
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=eaf96a83cff32661351004abaacd45228366a3bee8694940b103b090b74e5c3c
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built openpyxl docopt
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import records

In [ ]:

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = convert_to_unicode(reader.readline())
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab



In [ ]:
def convert_to_unicode(text):
    """Converts `text` to Unicode (if it's not already), assuming utf-8 input."""
    if six.PY3:
        if isinstance(text, str):
            return text
        elif isinstance(text, bytes):
            return text.decode("utf-8", "ignore")
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2:
        if isinstance(text, str):
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            return text
        else:
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else:
        raise ValueError("Not running on Python2 or Python 3?")



def convert_tokens_to_ids(vocab, tokens):
    """Converts a sequence of tokens into ids using the vocab."""
    ids = []
    for token in tokens:
        ids.append(vocab[token])
    return ids


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens




In [ ]:
class WordpieceTokenizer(object):
    """Runs WordPiece tokenization."""

    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        """Tokenizes a piece of text into its word pieces.

        This uses a greedy longest-match-first algorithm to perform tokenization
        using the given vocabulary.

        For example:
          input = "unaffable"
          output = ["un", "##aff", "##able"]

        Args:
          text: A single token or whitespace separated tokens. This should have
            already been passed through `BasicTokenizer.

        Returns:
          A list of wordpiece tokens.
        """

        text = convert_to_unicode(text)

        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end

            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


In [ ]:
class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self, do_lower_case=True):
        """Constructs a BasicTokenizer.

        Args:
          do_lower_case: Whether to lower case the input.
        """
        self.do_lower_case = do_lower_case

    def tokenize(self, text):
        """Tokenizes a piece of text."""
        text = convert_to_unicode(text)
        text = self._clean_text(text)

        text = self._tokenize_chinese_chars(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case:
                token = token.lower()
                token = self._run_strip_accents(token)
            split_tokens.extend(self._run_split_on_punc(token))

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def _tokenize_chinese_chars(self, text):
        """Adds whitespace around any CJK character."""
        output = []
        for char in text:
            cp = ord(char)
            if self._is_chinese_char(cp):
                output.append(" ")
                output.append(char)
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _is_chinese_char(self, cp):
        """Checks whether CP is the codepoint of a CJK character."""
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
            (cp >= 0x3400 and cp <= 0x4DBF) or  #
            (cp >= 0x20000 and cp <= 0x2A6DF) or  #
            (cp >= 0x2A700 and cp <= 0x2B73F) or  #
            (cp >= 0x2B740 and cp <= 0x2B81F) or  #
            (cp >= 0x2B820 and cp <= 0x2CEAF) or
            (cp >= 0xF900 and cp <= 0xFAFF) or  #
            (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
            return True

        return False

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

In [ ]:
class FullTokenizer(object):
    """Runs end-to-end tokenziation."""

    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_tokens_to_ids(self.vocab, tokens)


In [ ]:
tokenizer = FullTokenizer(vocab_file="/content/gdrive/MyDrive/vocab_uncased_L-12_H-768_A-12.txt", do_lower_case=True)


In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def get_wordnet_pos(token):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
path = '/content/gdrive/MyDrive/data/data'

def load_data(sql_paths, table_paths):
    if not isinstance(sql_paths, list):
        sql_paths = (sql_paths, )
    if not isinstance(table_paths, list):
        table_paths = (table_paths, )
    sql_data = []
    table_data = {}

    for SQL_PATH in sql_paths:
        print("Loading data from %s" % SQL_PATH)
        with open(SQL_PATH) as inf:
            for idx, line in enumerate(inf):
                sql = json.loads(line.strip())
                question = sql['question']
                question_tokens = tokenizer.tokenize(question.lower())
                lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token))
                    for token in question_tokens
                    if token.lower() not in stop_words and re.match(r'^[a-zA-Z0-9]+$', token)]
                final_lemmatized_tokens = []
                i = 0
                while i < len(lemmatized_tokens):
                    token = lemmatized_tokens[i]
                    if token.isdigit() and i + 1 < len(lemmatized_tokens) and lemmatized_tokens[i + 1].isdigit():
                        merged_token = token +'-'+ lemmatized_tokens[i + 1]
                        final_lemmatized_tokens.append(merged_token)
                        i += 2
                    else:
                        final_lemmatized_tokens.append(lemmatizer.lemmatize(token, get_wordnet_pos(token)))
                        i += 1
                sql['question_tok'] = final_lemmatized_tokens


                query = sql['sql']
                query_tokens = []
                query_tokens.append(query['sel'])
                query_tokens.append(query['agg'])

                col_cond_tok = []
                for cond in query['conds']:
                    col_tok = [cond[0]]
                    op_tok = [cond[1]]
                    value = str(cond[2])
                    value_tokens = tokenizer.tokenize(value.lower())
                    lemmatized_value_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in value_tokens]
                    col_cond_tok.append((col_tok, op_tok, lemmatized_value_tokens))

                sql['query_tok'] = (query_tokens, col_cond_tok)

                sql_data.append(sql)

    for TABLE_PATH in table_paths:
        print("Loading data from %s" % TABLE_PATH)
        with open(TABLE_PATH) as inf:
            for line in inf:
                tab = json.loads(line.strip())
                headers = tab['header']
                header_tokens = [tokenizer.tokenize(header.lower()) for header in headers]
                tab['header_tok'] = header_tokens
                table_data[tab['id']] = tab

    return sql_data, table_data

def load_dataset():
    print("Loading Data")
    sql_data, table_data = load_data([path + '/train.jsonl'], [path + '/train.tables.jsonl'])
    val_sql_data, val_table_data = load_data([path + '/dev.jsonl'], [path + '/dev.tables.jsonl'])
    test_sql_data, test_table_data = load_data([path + '/test.jsonl'], [path + '/test.tables.jsonl'])

    TRAIN_DB = path + '/train.db'
    DEV_DB = path + '/dev.db'
    TEST_DB = path + '/test.db'

    return (
        sql_data,
        table_data,
        val_sql_data,
        val_table_data,
        test_sql_data,
        test_table_data,
        TRAIN_DB,
        DEV_DB,
        TEST_DB
    )


In [ ]:
from google.colab import drive
import os


zip_path = '/content/gdrive/MyDrive/data.zip'

# Specify the destination directory for extracted files
destination_path = '/content/gdrive/MyDrive/data'

# Create the destination directory if it doesn't exist
os.makedirs(destination_path, exist_ok=True)

# Unzip the file to the destination directory
!unzip "{zip_path}" -d "{destination_path}"


Archive:  /content/gdrive/MyDrive/data.zip
replace /content/gdrive/MyDrive/data/data/dev.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
sql_paths = ['/content/gdrive/MyDrive/data/data/train.jsonl', '/content/gdrive/MyDrive/data/data/dev.jsonl', '/content/gdrive/MyDrive/data/data/test.jsonl']
table_paths = ['/content/gdrive/MyDrive/data/data/train.tables.jsonl', '/content/gdrive/MyDrive/data/data/dev.tables.jsonl', '/content/gdrive/MyDrive/data/data/test.tables.jsonl']
sql_data, table_data, val_sql_data, val_table_data, test_sql_data, test_table_data, TRAIN_DB, DEV_DB, TEST_DB = load_dataset()

Loading Data
Loading data from /content/gdrive/MyDrive/data/data/train.jsonl
Loading data from /content/gdrive/MyDrive/data/data/train.tables.jsonl
Loading data from /content/gdrive/MyDrive/data/data/dev.jsonl
Loading data from /content/gdrive/MyDrive/data/data/dev.tables.jsonl
Loading data from /content/gdrive/MyDrive/data/data/test.jsonl
Loading data from /content/gdrive/MyDrive/data/data/test.tables.jsonl


In [ ]:
train_question_text_list = [data['question']for data in sql_data]
val_question_text_list=[ data['question'] for data in val_sql_data]
train_column_names = []
for table_id, table_info in table_data.items():
    headers = table_info['header']
    train_column_names.append(headers)
val_column_names= []
for table_id, table_info in val_table_data.items():
    headers = table_info['header']
    val_column_names.append(headers)

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

classifier = pipeline("zero-shot-classification")

candidate_labels = ["NONE","MAX", "MIN", "COUNT", "SUM", "AVG"]
hypothesis_template = "This question is about {}."

train_inputs = [{'question': question, 'columns': columns, 'sql': sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(train_question_text_list, train_column_names))]

val_inputs = [{'question': question, 'columns': columns, 'sql': val_sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(val_question_text_list, val_column_names))]

# Batch size for training and validation
batch_size = 16

# Number of steps between saving checkpoints
save_interval = 150

# Training loop
train_results = []
num_train_batches = (len(train_inputs) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training")
for batch_start in range(0, len(train_inputs), batch_size):
    batch_end = min(batch_start + batch_size, len(train_inputs))
    batch_inputs = train_inputs[batch_start:batch_end]
    input_texts = [data['question'] for data in batch_inputs]
    input_columns = [" ".join(data['columns']) for data in batch_inputs]
    sql_attributes = [data['sql'] for data in batch_inputs]
    batch_results = classifier(input_texts, candidate_labels, input_columns=input_columns, hypothesis_template=hypothesis_template, sql_attributes=sql_attributes)
    train_results.extend(batch_results)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_{batch_start}.pt"
        torch.save(classifier.model.state_dict(), checkpoint_path)

    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop
val_results = []
num_val_batches = (len(val_inputs) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation")
for batch_start in range(0, len(val_inputs), batch_size):
    batch_end = min(batch_start + batch_size, len(val_inputs))
    batch_inputs = val_inputs[batch_start:batch_end]
    input_texts = [data['question'] for data in batch_inputs]
    input_columns = [" ".join(data['columns']) for data in batch_inputs]
    sql_attributes = [data['sql'] for data in batch_inputs]
    batch_results = classifier(input_texts, candidate_labels, input_columns=input_columns, hypothesis_template=hypothesis_template, sql_attributes=sql_attributes)
    val_results.extend(batch_results)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_{batch_start}.pt"
        torch.save(classifier.model.state_dict(), checkpoint_path)

    val_progress_bar.update(1)

val_progress_bar.close()


'''
from transformers import pipeline

classifier = pipeline("zero-shot-classification")

train_inputs = [{'question': question, 'columns': columns, 'sql': sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(train_question_text_list, train_column_names))]

val_inputs = [{'question': question, 'columns': columns, 'sql': val_sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(val_question_text_list, val_column_names))]

train_results = []
for input_data in train_inputs:
    input_text = input_data['question']
    input_columns = " ".join(input_data['columns'])
    agg_label = input_data['sql']['agg']
    result = classifier(input_text, [agg_label], input_columns=input_columns)
    train_results.append(result)

val_results = []
for input_data in val_inputs:
    input_text = input_data['question']
    input_columns = " ".join(input_data['columns'])
    agg_label = input_data['sql']['agg']
    result = classifier(input_text, [agg_label], input_columns=input_columns)
    val_results.append(result)
'''


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.

Training:  29%|██▉       | 36/125 [31:40<1:18:18, 52.79s/it]

Validation: 100%|██████████| 63/63 [49:56<00:00, 47.56s/it]


'\nfrom transformers import pipeline\n\nclassifier = pipeline("zero-shot-classification")\n\ntrain_inputs = [{\'question\': question, \'columns\': columns, \'sql\': sql_data[i][\'sql\']} for i, (question, columns) in enumerate(zip(train_question_text_list, train_column_names))]\n\nval_inputs = [{\'question\': question, \'columns\': columns, \'sql\': val_sql_data[i][\'sql\']} for i, (question, columns) in enumerate(zip(val_question_text_list, val_column_names))]\n\ntrain_results = []\nfor input_data in train_inputs:\n    input_text = input_data[\'question\']\n    input_columns = " ".join(input_data[\'columns\'])\n    agg_label = input_data[\'sql\'][\'agg\']\n    result = classifier(input_text, [agg_label], input_columns=input_columns)\n    train_results.append(result)\n\nval_results = []\nfor input_data in val_inputs:\n    input_text = input_data[\'question\']\n    input_columns = " ".join(input_data[\'columns\'])\n    agg_label = input_data[\'sql\'][\'agg\']\n    result = classifier(in

In [ ]:
classifier_path = "/content/gdrive/MyDrive/data/classifieragg.pt"
torch.save(classifier.model.state_dict(), classifier_path)

In [ ]:
test_input_texts = [
    "What is the maximum population?",
    "How many records are there in the table?",
    "What is the average salary of employees?",
]

test_input_columns = [
    "population",
    "records",
    "salary",
]

test_sql_attributes = [
    "SELECT MAX(population) FROM cities",
    "SELECT COUNT(*) FROM table_name",
    "SELECT AVG(salary) FROM employees",
]

# Perform zero-shot classification on the test inputs
test_results = classifier(
    test_input_texts,
    candidate_labels,
    input_columns=test_input_columns,
    hypothesis_template=hypothesis_template,
    sql_attributes=test_sql_attributes,
)

# Print the results
for i, result in enumerate(test_results):
    input_text = test_input_texts[i]
    print(f"Input Text: {input_text}")
    print(f"Predicted Label: {result['labels'][0]}")
    print(f"Score: {result['scores'][0]}")
    print("")

NameError: ignored

In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

classifier_column = pipeline("zero-shot-classification")

hypothesis_template_column = "This question is asking about the {} column."

train_inputs = [{'question': question, 'columns': columns} for question, columns in zip(train_question_text_list, train_column_names)]
val_inputs = [{'question': question, 'columns': columns} for question, columns in zip(val_question_text_list, val_column_names)]

# Batch size for training and validation
batch_size = 16

# Number of steps between saving checkpoints
save_interval = 20

# Collect candidate column names for each question in the training set
candidate_labels_columns_train = [data['columns'] for data in train_inputs]

# Training loop for column prediction
train_results_column = []
num_train_batches = (len(train_inputs) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training (Column prediction)")
for batch_start in range(0, len(train_inputs), batch_size):
    batch_end = min(batch_start + batch_size, len(train_inputs))
    batch_inputs = train_inputs[batch_start:batch_end]
    input_texts = [data['question'] for data in batch_inputs]
    candidate_labels_columns_batch = candidate_labels_columns_train[batch_start:batch_end]
    batch_results = classifier_column(input_texts, candidate_labels=candidate_labels_columns_batch, hypothesis_template=hypothesis_template_column)
    train_results_column.extend(batch_results)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_column_{batch_start}.pt"
        torch.save(classifier_column.model.state_dict(), checkpoint_path)

    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop for column prediction
candidate_labels_columns_val = [data['columns'] for data in val_inputs]
val_results_column = []
num_val_batches = (len(val_inputs) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation (Column prediction)")
for batch_start in range(0, len(val_inputs), batch_size):
    batch_end = min(batch_start + batch_size, len(val_inputs))
    batch_inputs = val_inputs[batch_start:batch_end]
    input_texts = [data['question'] for data in batch_inputs]
    candidate_labels_columns_batch = candidate_labels_columns_val[batch_start:batch_end]
    batch_results = classifier_column(input_texts, candidate_labels=candidate_labels_columns_batch, hypothesis_template=hypothesis_template_column)
    val_results_column.extend(batch_results)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_column_{batch_start}.pt"
        torch.save(classifier_column.model.state_dict(), checkpoint_path)

    val_progress_bar.update(1)

val_progress_bar.close()


preparing labels for where clause prediction

In [ ]:
# Training data
train_rows = []
for table_id, table_info in table_data.items():
    rows = table_info['rows']
    train_rows.append(rows)

train_inputs1 = []
for i, (question, columns) in enumerate(zip(train_question_text_list, train_column_names)):
    sql = sql_data[i]['sql']
    conditions = sql.get('conds', [])  # Get 'conds' from sql_data[i]['sql'] if available, otherwise use an empty list
    target_indexes_list = [(cond[0], cond[1], cond[2]) for cond in conditions]

    train_inputs1.append({
        'question': question,
        'columns': columns,
        'sql': sql,
        'target_indexes_list': target_indexes_list
    })

# Validation data
val_rows = []
for table_id, table_info in val_table_data.items():
    rows = table_info['rows']
    val_rows.append(rows)

val_inputs1 = []
for i, (question, columns) in enumerate(zip(val_question_text_list, val_column_names)):
    sql = val_sql_data[i]['sql']
    conditions = sql.get('conds', [])  # Get 'conds' from val_sql_data[i]['sql'] if available, otherwise use an empty list
    target_indexes_list = [(cond[0], cond[1], cond[2]) for cond in conditions]

    val_inputs1.append({
        'question': question,
        'columns': columns,
        'sql': sql,
        'target_indexes_list': target_indexes_list
    })

# Candidate labels
candidate_labels_columns_train = [data['columns'] for data in train_inputs1]
candidate_labels_operators = ["=", ">","<"]
candidate_labels_values = [str(item) for sublist in train_rows for item in sublist]


In [ ]:
import ast

# Assuming `candidate_label_values` is a list of strings, each containing a list of possible values
processed_candidate_labels = []

for value_string in candidate_labels_values:
    try:
        value_list = ast.literal_eval(value_string)
        # Process the list to extract the relevant information for zero-shot classification
        processed_label = " ".join(str(value) for value in value_list if value not in ["No slogan on current series", "New series began in June 2011", "Current series will be exhausted this year"])
        processed_candidate_labels.append(processed_label)
    except ValueError:
        # Handle any parsing errors
        continue

print(processed_candidate_labels)


['Australian Capital Territory blue/white Yaa·nna ACT · CELEBRATION OF A CENTURY 2013 YIL·00A Slogan screenprinted on plate', 'New South Wales black/yellow aa·nn·aa NEW SOUTH WALES BX·99·HI', 'New South Wales black/white aaa·nna NSW CPX·12A Optional white slimline series', 'Northern Territory ochre/white Ca·nn·aa NT · OUTBACK AUSTRALIA CB·06·ZZ', 'Queensland maroon/white nnn·aaa QUEENSLAND · SUNSHINE STATE 999·TLG Slogan embossed on plate', 'South Australia black/white Snnn·aaa SOUTH AUSTRALIA S000·AZD', 'Victoria blue/white aaa·nnn VICTORIA - THE PLACE TO BE ZZZ·562', 'Robinson R-22 Light utility helicopter 1,370 lb (635 kg) 497 ft² (46.2 m²) 2.6 lb/ft² (14 kg/m²)', 'Bell 206B3 JetRanger Turboshaft utility helicopter 3,200 lb (1,451 kg) 872 ft² (81.1 m²) 3.7 lb/ft² (18 kg/m²)', 'CH-47D Chinook Tandem rotor helicopter 50,000 lb (22,680 kg) 5,655 ft² (526 m²) 8.8 lb/ft² (43 kg/m²)', 'Mil Mi-26 Heavy-lift helicopter 123,500 lb (56,000 kg) 8,495 ft² (789 m²) 14.5 lb/ft² (71 kg/m²)', 'CH-5

In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Load the zero-shot classifiers for operator prediction
classifier_operator = pipeline("zero-shot-classification")

# Hypothesis template for the operator classifier
hypothesis_template_operator = "This question is asking for the {} operator."

# Assuming you have the following data structure for the train and validation sets
train_data = train_inputs1  # Assuming train_inputs is your list of training data
val_data = val_inputs1  # Assuming val_inputs is your list of validation data

# Batch size for training and validation
batch_size = 16
# Training loop for the operator classifier
train_results_operator = []
num_train_batches = (len(train_data) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training Operator Classifier")
for batch_start in range(0, len(train_data), batch_size):
    batch_end = min(batch_start + batch_size, len(train_data))
    batch_data = train_data[batch_start:batch_end]
    input_texts_operator = [data['question'] for data in batch_data]

    # Classifier for operator prediction
    batch_results_operator = classifier_operator(input_texts_operator, candidate_labels_operators, hypothesis_template=hypothesis_template_operator)

    # Combine the results from the operator classifier and the target information
    for i, result_operator in enumerate(batch_results_operator):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                operator_index = condition_data[1]
                result = {
                    'question': batch_data[i]['question'],
                    'operator_prediction': result_operator['labels'][operator_index]
                }
                train_results_operator.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop for the operator classifier
val_results_operator = []
num_val_batches = (len(val_data) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation Operator Classifier")
for batch_start in range(0, len(val_data), batch_size):
    batch_end = min(batch_start + batch_size, len(val_data))
    batch_data = val_data[batch_start:batch_end]
    input_texts_operator = [data['question'] for data in batch_data]

    # Classifier for operator prediction
    batch_results_operator = classifier_operator(input_texts_operator, candidate_labels_operators, hypothesis_template=hypothesis_template_operator)

    # Combine the results from the operator classifier and the target information
    for i, result_operator in enumerate(batch_results_operator):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                operator_index = condition_data[1]
                result = {
                    'question': batch_data[i]['question'],
                    'operator_prediction': result_operator['labels'][operator_index]
                }
                val_results_operator.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    val_progress_bar.update(1)

val_progress_bar.close()

# Save the trained operator model
classifier_path_operator = "/content/gdrive/MyDrive/data/classifierwhereop.pt"
torch.save(classifier_operator.model.state_dict(), classifier_path_operator)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Validation Operator Classifier: 100%|██████████| 35/35 [16:33<00:00, 28.37s/it]


In [ ]:
# Test questions
test_input_texts = [
    "What is the sum of 10 and 20?",
    "Find all items greater than $50.",
    "Calculate 5 times 6.",
    "Show values equal to 100.",
    "Find all records where the price is less than $500.",
    "Find the number of employees with age greater than 40.",
]

# Test SQL attributes
test_sql_attributes = [
    "SELECT 10 + 20",
    "SELECT * FROM items_table WHERE price > 50",
    "SELECT 5 * 6",
    "SELECT * FROM table_name WHERE value = 100",
    "SELECT * FROM records_table WHERE price < 500",
    "SELECT COUNT(*) FROM employees_table WHERE age > 40",
]
# Perform zero-shot classification on the test inputs
test_results_operator = []
for i, input_text in enumerate(test_input_texts):
    # Classifier for operator prediction
    result_operator = classifier_operator(input_text, candidate_labels_operators, hypothesis_template=hypothesis_template_operator)

    # Get the predicted operator
    predicted_operator = result_operator['labels'][0]

    # Combine the input text, predicted operator, and SQL attribute
    result = {
        'input_text': input_text,
        'predicted_operator': predicted_operator,
        'sql_attribute': test_sql_attributes[i],
    }
    test_results_operator.append(result)

# Print the results
for result in test_results_operator:
    input_text = result['input_text']
    predicted_operator = result['predicted_operator']
    sql_attribute = result['sql_attribute']

    print(f"Input Text: {input_text}")
    print(f"Predicted Operator: {predicted_operator}")
    print(f"SQL Attribute: {sql_attribute}")
    print("")


Input Text: What is the sum of 10 and 20?
Predicted Operator: >
SQL Attribute: SELECT 10 + 20

Input Text: Find all items greater than $50.
Predicted Operator: >
SQL Attribute: SELECT * FROM items_table WHERE price > 50

Input Text: Calculate 5 times 6.
Predicted Operator: =
SQL Attribute: SELECT 5 * 6

Input Text: Show values equal to 100.
Predicted Operator: =
SQL Attribute: SELECT * FROM table_name WHERE value = 100

Input Text: Find all records where the price is less than $500.
Predicted Operator: <
SQL Attribute: SELECT * FROM records_table WHERE price < 500

Input Text: Find the number of employees with age greater than 40.
Predicted Operator: =
SQL Attribute: SELECT COUNT(*) FROM employees_table WHERE age > 40



In [ ]:
# Load the zero-shot classifiers for value prediction
classifier_value = pipeline("zero-shot-classification")

# Hypothesis template for the value classifier
hypothesis_template_value = "This question is asking for the value {}."

# Training loop for the value classifier
train_results_value = []
num_train_batches = (len(train_data) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training Value Classifier")
for batch_start in range(0, len(train_data), batch_size):
    batch_end = min(batch_start + batch_size, len(train_data))
    batch_data = train_data[batch_start:batch_end]
    input_texts_value = [data['question'] for data in batch_data]

    # Classifier for value prediction
    batch_results_value = classifier_value(input_texts_value, candidate_labels_values, hypothesis_template=hypothesis_template_value)

    # Combine the results from the value classifier and the target information
    for i, result_value in enumerate(batch_results_value):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                value_index = condition_data[2]
                result = {
                    'question': batch_data[i]['question'],
                    'value_prediction': result_value['labels'][value_index]
                }
                train_results_value.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop for the value classifier
val_results_value = []
num_val_batches = (len(val_data) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation Value Classifier")
for batch_start in range(0, len(val_data), batch_size):
    batch_end = min(batch_start + batch_size, len(val_data))
    batch_data = val_data[batch_start:batch_end]
    input_texts_value = [data['question'] for data in batch_data]

    # Classifier for value prediction
    batch_results_value = classifier_value(input_texts_value, candidate_labels_values, hypothesis_template=hypothesis_template_value)

    # Combine the results from the value classifier and the target information
    for i, result_value in enumerate(batch_results_value):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                value_index = condition_data[2]
                result = {
                    'question': batch_data[i]['question'],
                    'value_prediction': result_value['labels'][value_index]
                }
                val_results_value.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    val_progress_bar.update(1)

val_progress_bar.close()


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Training Value Classifier:   0%|          | 0/58 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Load the "zero-shot-classification" pipeline for column prediction
classifier_column = pipeline("zero-shot-classification")

# Hypothesis template for the column classifier
hypothesis_template_column = "This question is asking for the {} column."

# Assuming you have the following data structure for the train and validation sets
train_data = train_inputs1  # Assuming train_inputs1 is your list of training data
val_data = val_inputs1  # Assuming val_inputs1 is your list of validation data

# Batch size for training and validation
batch_size = 16

# Training loop for the column classifier
train_results_column = []
num_train_batches = (len(train_data) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training Column Classifier")
for batch_start in range(0, len(train_data), batch_size):
    batch_end = min(batch_start + batch_size, len(train_data))
    batch_data = train_data[batch_start:batch_end]
    input_texts_column = [data['question'] for data in batch_data]

    # Classifier for column prediction
    batch_results_column = classifier_column(input_texts_column, hypothesis_template=hypothesis_template_column)

    # Combine the results from the column classifier and the target information
    for i, result_column in enumerate(batch_results_column):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                column_index = condition_data[0]
                result = {
                    'question': batch_data[i]['question'],
                    'column_prediction': result_column['labels'][column_index]
                }
                train_results_column.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop for the column classifier
val_results_column = []
num_val_batches = (len(val_data) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation Column Classifier")
for batch_start in range(0, len(val_data), batch_size):
    batch_end = min(batch_start + batch_size, len(val_data))
    batch_data = val_data[batch_start:batch_end]
    input_texts_column = [data['question'] for data in batch_data]

    # Classifier for column prediction
    batch_results_column = classifier_column(input_texts_column, hypothesis_template=hypothesis_template_column)

    # Combine the results from the column classifier and the target information
    for i, result_column in enumerate(batch_results_column):
        if batch_data[i]['target_indexes_list']:
            for condition_data in batch_data[i]['target_indexes_list']:
                column_index = condition_data[0]
                result = {
                    'question': batch_data[i]['question'],
                    'column_prediction': result_column['labels'][column_index]
                }
                val_results_column.append(result)

    # Clear GPU memory
    torch.cuda.empty_cache()
    val_progress_bar.update(1)

val_progress_bar.close()

# Save the trained column model
classifier_path_column = "/content/gdrive/MyDrive/data/classifierwherecolumn.pt"
torch.save(classifier_column.model.state_dict(), classifier_path_column)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Training Column Classifier:   0%|          | 0/58 [00:00<?, ?it/s]

TypeError: ignored

In [ ]:

target_operators_list = []
target_values_list = []
for data in train_inputs:
    condition_list = data['target_indexes_list']
    operators = [condition[1] for condition in condition_list]
    values = [condition[2] for condition in condition_list]
    target_operators_list.append(operators)
    target_values_list.append(values)


In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Load the zero-shot classifiers for column, operator, and value predictions
classifier_column = pipeline("zero-shot-classification")
classifier_operator = pipeline("zero-shot-classification")
classifier_value = pipeline("zero-shot-classification")

# Hypothesis templates for each sub-classifier
hypothesis_template_column = "This question is asking about the {} column."
hypothesis_template_operator = "This question is asking for the {} operator."
hypothesis_template_value = "This question is asking for the value {}."

# Assuming you have the following data structure for the train and validation sets
train_data = train_inputs1  # Assuming train_inputs is your list of training data
val_data = val_inputs1  # Assuming val_inputs is your list of validation data

# Batch size for training and validation
batch_size = 16



# Training loop
train_results = []
num_train_batches = (len(train_data) + batch_size - 1) // batch_size
train_progress_bar = tqdm(total=num_train_batches, desc="Training")
for batch_start in range(0, len(train_data), batch_size):
    batch_end = min(batch_start + batch_size, len(train_data))
    batch_data = train_data[batch_start:batch_end]
    input_texts_column = [ data['question'] for data in batch_data]
    input_texts_operator = [ data['question'] for data in batch_data]
    input_texts_value = [  data['question'] for data in batch_data]

    # Classifier for column prediction
    batch_results_column = classifier_column(input_texts_column, candidate_labels_columns_train, hypothesis_template=hypothesis_template_column)
    # Classifier for operator prediction
    batch_results_operator = classifier_operator(input_texts_operator, candidate_labels_operators, hypothesis_template=hypothesis_template_operator)
    # Classifier for value prediction
    batch_results_value = classifier_value(input_texts_value, candidate_labels_values, hypothesis_template=hypothesis_template_value)

    # Combine the results from the three classifiers and the target information
    for i, (result_column, result_operator, result_value) in enumerate(zip(batch_results_column, batch_results_operator, batch_results_value)):
        # Check if target_indexes_list is empty
        if batch_data[i]['target_indexes_list']:
            # Obtain target labels using target indexes for each classifier
            column_index, _, column_name = batch_data[i]['target_indexes_list'][0]
            operator_index, _, operator_label = batch_data[i]['target_indexes_list'][1]
            value_index, _, value_label = batch_data[i]['target_indexes_list'][2]

            result = {
                'question': batch_data[i]['question'],
                'column_prediction': result_column['labels'][column_index],
                'operator_prediction': result_operator['labels'][operator_index],
                'value_prediction': result_value['labels'][value_index]
            }

            train_results.append(result)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_{batch_start}.pt"
        torch.save(classifier_column.model.state_dict(), checkpoint_path)

    train_progress_bar.update(1)

train_progress_bar.close()

# Validation loop
val_results = []
num_val_batches = (len(val_data) + batch_size - 1) // batch_size
val_progress_bar = tqdm(total=num_val_batches, desc="Validation")
for batch_start in range(0, len(val_data), batch_size):
    batch_end = min(batch_start + batch_size, len(val_data))
    batch_data = val_data[batch_start:batch_end]
    input_texts_column = [ data['question'] for data in batch_data]
    input_texts_operator = [  data['question'] for data in batch_data]
    input_texts_value = [ data['question'] for data in batch_data]

    # Classifier for column prediction
    batch_results_column = classifier_column(input_texts_column, candidate_labels_columns_train, hypothesis_template=hypothesis_template_column)
    # Classifier for operator prediction
    batch_results_operator = classifier_operator(input_texts_operator, candidate_labels_operators, hypothesis_template=hypothesis_template_operator)
    # Classifier for value prediction
    batch_results_value = classifier_value(input_texts_value, candidate_labels_values, hypothesis_template=hypothesis_template_value)

    # Combine the results from the three classifiers and the target information
    for i, (result_column, result_operator, result_value) in enumerate(zip(batch_results_column, batch_results_operator, batch_results_value)):
        # Check if target_indexes_list is empty
        if batch_data[i]['target_indexes_list']:
            # Obtain target labels using target indexes for each classifier
            column_index, _, column_name = batch_data[i]['target_indexes_list'][0]
            operator_index, _, operator_label = batch_data[i]['target_indexes_list'][1]
            value_index, _, value_label = batch_data[i]['target_indexes_list'][2]

            result = {
                'question': batch_data[i]['question'],
                'column_prediction': result_column['labels'][column_index],
                'operator_prediction': result_operator['labels'][operator_index],
                'value_prediction': result_value['labels'][value_index]
            }

            val_results.append(result)

    # Check if it's time to save a checkpoint
    if batch_start % save_interval == 0 and batch_start > 0:
        checkpoint_path = f"checkpoint_{batch_start}.pt"
        torch.save(classifier_column.model.state_dict(), checkpoint_path)

    val_progress_bar.update(1)

val_progress_bar.close()
classifier_path = "/content/gdrive/MyDrive/data/classifierwherecol.pt"
torch.save(classifier_column.model.state_dict(), classifier_path)

classifier_path1 = "/content/gdrive/MyDrive/data/classifierwhereop.pt"
torch.save(classifier_column.model.state_dict(), classifier_path)

classifier_path2 = "/content/gdrive/MyDrive/data/classifierwhereval.pt"
torch.save(classifier_column.model.state_dict(), classifier_path)


No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Training:   0%|          | 0/58 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline
import torch
from tqdm import tqdm

class SQLNet(nn.Module):
    def __init__(self, word_emb, N_word, N_h=100, N_depth=2, gpu=False):
        super(SQLNet, self).__init__()

        # Your existing __init__ code
        self.gpu = gpu
        self.N_h = N_h
        self.N_depth = N_depth
        self.max_col_num = 45
        self.max_tok_num = 200
        self.SQL_TOK = ['<UNK>', '<END>', 'WHERE', 'AND', 'EQL', 'GT', 'LT', '<BEG>']
        self.COND_OPS = ['EQL', 'GT', 'LT']
        self.embed_layer = WordEmbedding(word_emb, N_word)

        # Load the zero-shot classifier for column selection prediction
        self.classifier_column = pipeline("zero-shot-classification")

        # Add your aggregation operator prediction model and classifiers for where clause here
        self.agg_pred_model = ()
        self.classifier_where_col = ()
        self.classifier_where_op = ()
        self.classifier_where_val = ()

        # Your existing code for other modules (AggPredictor, SelPredictor, SQLNetCondPredictor)

    def forward(self, q, col, col_num, pred_entry, gt_where=None, gt_cond=None, reinforce=False, gt_sel=None):
        B = len(q)
        pred_agg, pred_sel, pred_cond = pred_entry

        agg_score = None
        sel_score = None
        cond_score = None

        x_emb_var, x_len = self.embed_layer.gen_x_batch(q)
        col_inp_var, col_name_len, col_len = self.embed_layer.gen_col_batch(col)

        max_x_len = max(x_len)

        if pred_agg:
            agg_score = self.agg_pred_model(x_emb_var, x_len, col_inp_var, col_name_len, col_len, col_num, gt_sel=gt_sel)

        if pred_sel:
            sel_score = self.zero_shot_column_classification(q, col)

        if pred_cond:
            cond_score_col = self.classifier_where_col(x_emb_var, x_len, col_inp_var, col_name_len, col_len, col_num)
            cond_score_op = self.classifier_where_op(x_emb_var, x_len, col_inp_var, col_name_len, col_len, col_num)
            cond_score_val = self.classifier_where_val(x_emb_var, x_len, col_inp_var, col_name_len, col_len, col_num)

            # Combine the results from the three classifiers and the target information
            cond_score = (cond_score_col, cond_score_op, cond_score_val)

        return (agg_score, sel_score, cond_score)

    def zero_shot_column_classification(self, questions, columns):

        candidate_labels_columns = columns
        input_texts = questions
        batch_results = self.classifier_column(input_texts, candidate_labels=candidate_labels_columns, hypothesis_template="This question is asking about the {} column.")
        return batch_results
    def forward(self, q, col, col_num, pred_entry,gt_where = None, gt_cond=None, reinforce=False, gt_sel=None):
        B = len(q)
        pred_agg, pred_sel, pred_cond = pred_entry

        agg_score = None
        sel_score = None
        cond_score = None

        #Predict aggregator

        x_emb_var, x_len = self.embed_layer.gen_x_batch(q)
        #x_emb_var : un array contenant pour chaque question , ses words embeddings
        #x_len :une liste contenant les longueur des questions + <BEG>+<END> (chaque question +2)

        col_inp_var, col_name_len, col_len = self.embed_layer.gen_col_batch(col)
        #col_inp_var : un array contenant pour chaque colonne ses words embedding
        #col_name_len: nombre de mot pour chaque colonne
        #col_len: nombre de colonne pour chaque question

        max_x_len = max(x_len)#nombre de mots maximale dans les questions

        #comprendre le agg_score
        if pred_agg:
                agg_score = self.agg_pred(x_emb_var, x_len, col_inp_var,col_name_len, col_len, col_num, gt_sel=gt_sel)
                #print('agg_score : ',agg_score)

        if pred_sel:
                sel_score = self.sel_pred(x_emb_var, x_len, col_inp_var,col_name_len, col_len, col_num)
                #print('sel_score',sel_score)

        if pred_cond:
                cond_score = self.cond_pred(x_emb_var, x_len, col_inp_var,col_name_len, col_len, col_num,gt_where, gt_cond, reinforce=reinforce)
                #print('cond_score',cond_score)

        return (agg_score, sel_score, cond_score)

    def loss(self, score, truth_num, pred_entry, gt_where):
        pred_agg, pred_sel, pred_cond = pred_entry
        agg_score, sel_score, cond_score = score

        loss = 0
        if pred_agg:
            agg_truth = map(lambda x:x[0], truth_num)
            data = torch.from_numpy(np.array(agg_truth))
            if self.gpu:
                agg_truth_var = Variable(data.cuda())
            else:
                agg_truth_var = Variable(data)

            loss += self.CE(agg_score, agg_truth_var)

        if pred_sel:
            sel_truth = map(lambda x:x[1], truth_num)
            data = torch.from_numpy(np.array(sel_truth))
            if self.gpu:
                sel_truth_var = Variable(data.cuda())
            else:
                sel_truth_var = Variable(data)

            loss += self.CE(sel_score, sel_truth_var)

        if pred_cond:
            B = len(truth_num)
            cond_num_score, cond_col_score,\
                    cond_op_score, cond_str_score = cond_score
            #Evaluate the number of conditions
            cond_num_truth = map(lambda x:x[2], truth_num)
            data = torch.from_numpy(np.array(cond_num_truth))
            if self.gpu:
                cond_num_truth_var = Variable(data.cuda())
            else:
                cond_num_truth_var = Variable(data)
            loss += self.CE(cond_num_score, cond_num_truth_var)

            #Evaluate the columns of conditions
            T = len(cond_col_score[0])
            truth_prob = np.zeros((B, T), dtype=np.float32)
            for b in range(B):
                if len(truth_num[b][3]) > 0:
                    truth_prob[b][list(truth_num[b][3])] = 1
            data = torch.from_numpy(truth_prob)
            if self.gpu:
                cond_col_truth_var = Variable(data.cuda())
            else:
                cond_col_truth_var = Variable(data)

            sigm = nn.Sigmoid()
            cond_col_prob = sigm(cond_col_score)
            bce_loss = -torch.mean( 3*(cond_col_truth_var * \
                    torch.log(cond_col_prob+1e-10)) + \
                    (1-cond_col_truth_var) * torch.log(1-cond_col_prob+1e-10) )
            loss += bce_loss

            #Evaluate the operator of conditions
            for b in range(len(truth_num)):
                if len(truth_num[b][4]) == 0:
                    continue
                data = torch.from_numpy(np.array(truth_num[b][4]))
                if self.gpu:
                    cond_op_truth_var = Variable(data.cuda())
                else:
                    cond_op_truth_var = Variable(data)
                cond_op_pred = cond_op_score[b, :len(truth_num[b][4])]
                loss += (self.CE(cond_op_pred, cond_op_truth_var) \
                        / len(truth_num))

            #Evaluate the strings of conditions
            for b in range(len(gt_where)):
                for idx in range(len(gt_where[b])):
                    cond_str_truth = gt_where[b][idx]
                    if len(cond_str_truth) == 1:
                        continue
                    data = torch.from_numpy(np.array(cond_str_truth[1:]))
                    if self.gpu:
                        cond_str_truth_var = Variable(data.cuda())
                    else:
                        cond_str_truth_var = Variable(data)
                    str_end = len(cond_str_truth)-1
                    cond_str_pred = cond_str_score[b, idx, :str_end]
                    loss += (self.CE(cond_str_pred, cond_str_truth_var) \
                            / (len(gt_where) * len(gt_where[b])))

        return loss

    def check_acc(self, vis_info, pred_queries, gt_queries, pred_entry):
        def pretty_print(vis_data):
            print ('question:', vis_data[0])
            print ('headers: (%s)'%(' || '.join(vis_data[1])))
            print ('query:', vis_data[2])

        def gen_cond_str(conds, header):
            if len(conds) == 0:
                return 'None'
            cond_str = []
            for cond in conds:
                cond_str.append(header[cond[0]] + ' ' +
                    self.COND_OPS[cond[1]] + ' ' + unicode(cond[2]).lower())
            return 'WHERE ' + ' AND '.join(cond_str)

        pred_agg, pred_sel, pred_cond = pred_entry

        B = len(gt_queries)

        tot_err = agg_err = sel_err = cond_err = 0.0
        cond_num_err = cond_col_err = cond_op_err = cond_val_err = 0.0
        agg_ops = ['None', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
        for b, (pred_qry, gt_qry) in enumerate(zip(pred_queries, gt_queries)):
            good = True
            if pred_agg:
                agg_pred = pred_qry['agg']
                agg_gt = gt_qry['agg']
                if agg_pred != agg_gt:
                    agg_err += 1
                    good = False

            if pred_sel:
                sel_pred = pred_qry['sel']
                sel_gt = gt_qry['sel']
                if sel_pred != sel_gt:
                    sel_err += 1
                    good = False

            if pred_cond:
                cond_pred = pred_qry['conds']
                cond_gt = gt_qry['conds']
                flag = True
                if len(cond_pred) != len(cond_gt):
                    flag = False
                    cond_num_err += 1

                if flag and set(x[0] for x in cond_pred) != \
                        set(x[0] for x in cond_gt):
                    flag = False
                    cond_col_err += 1

                for idx in range(len(cond_pred)):
                    if not flag:
                        break
                    gt_idx = tuple(
                            x[0] for x in cond_gt).index(cond_pred[idx][0])
                    if flag and cond_gt[gt_idx][1] != cond_pred[idx][1]:
                        flag = False
                        cond_op_err += 1

                for idx in range(len(cond_pred)):
                    if not flag:
                        break
                    gt_idx = tuple(
                            x[0] for x in cond_gt).index(cond_pred[idx][0])
                    if flag and unicode(cond_gt[gt_idx][2]).lower() != \
                            unicode(cond_pred[idx][2]).lower():
                        flag = False
                        cond_val_err += 1

                if not flag:
                    cond_err += 1
                    good = False

            if not good:
                tot_err += 1

        return np.array((agg_err, sel_err, cond_err)), tot_err
    def gen_query(self, score, q, col, raw_q, raw_col,
            pred_entry, reinforce=False, verbose=False):
        def merge_tokens(tok_list, raw_tok_str):
            tok_str = raw_tok_str.lower()
            alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789$('
            special = {'-LRB-':'(',
                    '-RRB-':')',
                    '-LSB-':'[',
                    '-RSB-':']',
                    '``':'"',
                    '\'\'':'"',
                    '--':u'\u2013'}
            ret = ''
            double_quote_appear = 0
            for raw_tok in tok_list:
                if not raw_tok:
                    continue
                tok = special.get(raw_tok, raw_tok)
                if tok == '"':
                    double_quote_appear = 1 - double_quote_appear

                if len(ret) == 0:
                    pass
                elif len(ret) > 0 and ret + ' ' + tok in tok_str:
                    ret = ret + ' '
                elif len(ret) > 0 and ret + tok in tok_str:
                    pass
                elif tok == '"':
                    if double_quote_appear:
                        ret = ret + ' '
                elif tok[0] not in alphabet:
                    pass
                elif (ret[-1] not in ['(', '/', u'\u2013', '#', '$', '&']) \
                        and (ret[-1] != '"' or not double_quote_appear):
                    ret = ret + ' '
                ret = ret + tok
            return ret.strip()

        pred_agg, pred_sel, pred_cond = pred_entry
        agg_score, sel_score, cond_score = score

        ret_queries = []
        if pred_agg:
            B = len(agg_score)
        elif pred_sel:
            B = len(col)
        elif pred_cond:
            B = len(col)
        for b in range(B):
            cur_query = {}
            if pred_agg:
                cur_query['agg'] = np.argmax(agg_score[b].data.cpu().numpy())
            if pred_sel:
              if (B==1):
                cur_query['sel'] = np.argmax(sel_score.data.cpu().numpy())
              elif (B>1):
                cur_query['sel'] = np.argmax(sel_score[b].data.cpu().numpy())
            if pred_cond:
                cur_query['conds'] = []
                cond_num_score,cond_col_score,cond_op_score,cond_str_score =\
                        [x.data.cpu().numpy() for x in cond_score]
                cond_num = np.argmax(cond_num_score)
                all_toks = ['<BEG>'] + q[b] + ['<END>']
                max_idxes = np.argsort(-cond_col_score)[:cond_num]
                for idx in range(cond_num):
                    cur_cond = []
                    cur_cond.append(max_idxes[idx])
                    cur_cond.append(np.argmax(cond_op_score[idx]))
                    cur_cond_str_toks = []
                    for str_score in cond_str_score[idx]:

                        str_tok = np.argmax(str_score[:len(all_toks)])
                        str_val = all_toks[str_tok]
                        if str_val == '<END>':
                            break
                        cur_cond_str_toks.append(str_val)
                    cur_cond.append(merge_tokens(cur_cond_str_toks, raw_q[b]))
                    cur_query['conds'].append(cur_cond)
            ret_queries.append(cur_query)

        return ret_queries

preparing output of sub_models as input for the SQLNET model

In [ ]:
from transformers import BertModel,BertTokenizer
train_inputs = [{'question': question, 'columns': columns, 'sql': sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(train_question_text_list, train_column_names))]
val_inputs = [{'question': question, 'columns': columns, 'sql': val_sql_data[i]['sql']} for i, (question, columns) in enumerate(zip(val_question_text_list, val_column_names))]
checkpoint='bert_uncased_L-12_H-768_A-12'
tokenizer=BertTokenizer.from_pretrained(checkpoint)
configfile='/content/gdrive/MyDrive/bert_config_uncased_L-12_H-768_A-12.json'
model=BertModel.from_pretrained(checkpoint)
model(**train_inputs)


#to implement :
#custom loss function
#custum attention mechanism
